# This project was made for learning purposes. In making this, I followed [the official TensorFlow tutorial](https://www.tensorflow.org/tutorials/text/text_generation).

### This tutorial demonstrates how to generate text using a character-based RNN. 
We will work with a dataset of Shakespeare's writing from Andrej Karpathy's The Unreasonable Effectiveness of Recurrent Neural Networks. Given a sequence of characters from this data ("Shakespear"), train a model to predict the next character in the sequence ("e"). Longer sequences of text can be generated by calling the model repeatedly.


While some of the sentences are grammatical, most do not make sense. The model has not learned the meaning of words, but consider:

    - The model is character-based. When training started, the model did not know how to spell an English word, or that words were even a unit of text.

    - The structure of the output resembles a play—blocks of text generally begin with a speaker name, in all capital letters similar to the dataset.

    - As demonstrated below, the model is trained on small batches of text (100 characters each), and is still able to generate a longer sequence of text with coherent structure.


In [1]:
import tensorflow as tf
import numpy as np
import os, time

In [2]:
#if you dont not have the tensorflow module installed, uncomment the command below and run it
#!pip install tensorflow

### Load and preprocess the text

In [3]:
path_to_file = tf.keras.utils.get_file('shakespeare.txt', 'https://raw.githubusercontent.com/MartynaIwaniec/songs-AI/master/pop-songs.txt')

text = open(path_to_file, 'rb').read().decode(encoding='utf-8')

print ('Length of text: {} characters'.format(len(text)))
print("\n")

#a sample
print(text[:250])

Length of text: 1115394 characters


First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.



In [4]:
vocab = sorted(set(text))
print ('There is {} unique characters in the text'.format(len(vocab)))

There is 65 unique characters in the text


### Processing the text

### Mapping strings to numerical representations:
### 1. map chars to nrs
### 2. map nrs to chars

In [5]:
# creating a mapping from unique characters to indices
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

text_as_int = np.array([char2idx[c] for c in text])

In [6]:
# now we have an integer representation for each character.
# notice that we mapped the character as indexes from 0 to len(unique).

print('{')
for char,_ in zip(char2idx, range(20)):
    print('  {:4s}: {:3d},'.format(repr(char), char2idx[char]))
print('  ...\n}')

{
  '\n':   0,
  ' ' :   1,
  '!' :   2,
  '$' :   3,
  '&' :   4,
  "'" :   5,
  ',' :   6,
  '-' :   7,
  '.' :   8,
  '3' :   9,
  ':' :  10,
  ';' :  11,
  '?' :  12,
  'A' :  13,
  'B' :  14,
  'C' :  15,
  'D' :  16,
  'E' :  17,
  'F' :  18,
  'G' :  19,
  ...
}


In [7]:
# show how the first 13 characters from the text are mapped to integers
print ('{} ---- characters mapped to int ----> {}'.format(repr(text[:13]), text_as_int[:13]))

'First Citizen' ---- characters mapped to int ----> [18 47 56 57 58  1 15 47 58 47 64 43 52]


### Training the model for prediction task

#### Now, we have to train the model to guess what character (or a sequence of characters) is most likely to appear.

input = sequence of characters
output = a following character will be predicted

Because RNN depends on previously seen elements, given all the characters computed until this moment, what is the next character?

### Create training examples and targets

Next divide the text into example sequences. Each input sequence will contain **seq_length characters** from the text.

For each input sequence, the corresponding targets contain the same length of text, except shifted one character to the right.

So break the text into chunks of **seq_length+1**. 

E.g.
seq_length = 4
text = "Hello"
The input sequence would be *"Hell"*, and the target sequence *"ello"*.

To do this first use the **tf.data.Dataset.from_tensor_slices function** to convert the text vector into a stream of character indices.

In [8]:
# the max length sentence we want for a single input in characters
seq_length = 100
examples_per_epoch = len(text)//(seq_length+1)

# Create training examples / targets
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

for i in char_dataset.take(5):
    print(idx2char[i.numpy()])


F
i
r
s
t


In [9]:
# the batch method lets us easily convert these individual characters to sequences of the desired size

sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

for item in sequences.take(5):
    print(repr(''.join(idx2char[item.numpy()])))



'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '
'are all resolved rather to die than to famish?\n\nAll:\nResolved. resolved.\n\nFirst Citizen:\nFirst, you k'
"now Caius Marcius is chief enemy to the people.\n\nAll:\nWe know't, we know't.\n\nFirst Citizen:\nLet us ki"
"ll him, and we'll have corn at our own price.\nIs't a verdict?\n\nAll:\nNo more talking on't; let it be d"
'one: away, away!\n\nSecond Citizen:\nOne word, good citizens.\n\nFirst Citizen:\nWe are accounted poor citi'


#### For each sequence, duplicate and shift it to form the input and target text by using the map method to apply a simple function to each batch:
    

In [10]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)


In [11]:
# print the first examples input and target values:

for input_example, target_example in  dataset.take(1):
    print ('Input data: ', repr(''.join(idx2char[input_example.numpy()])))
    print ('Target data:', repr(''.join(idx2char[target_example.numpy()])))


Input data:  'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou'
Target data: 'irst Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '


#### Each index of these vectors are processed as one time step

#### For the input at time step 0, the model receives the index for "F" and trys to predict the index for "i" as the next character

#### At the next timestep, it does the same thing but the RNN considers the previous step context in addition to the current input character

In [12]:
for i, (input_idx, target_idx) in enumerate(zip(input_example[:5], target_example[:5])):
    print("Step {:4d}".format(i))
    print("  input: {} ({:s})".format(input_idx, repr(idx2char[input_idx])))
    print("  expected output: {} ({:s})".format(target_idx, repr(idx2char[target_idx])))

Step    0
  input: 18 ('F')
  expected output: 47 ('i')
Step    1
  input: 47 ('i')
  expected output: 56 ('r')
Step    2
  input: 56 ('r')
  expected output: 57 ('s')
Step    3
  input: 57 ('s')
  expected output: 58 ('t')
Step    4
  input: 58 ('t')
  expected output: 1 (' ')


### Create training batches:

#### We used tf.data to split the text into manageable sequences

#### But before feeding this data into the model, we need to shuffle the data and pack it into batches.

In [13]:
# Batch size
BATCH_SIZE = 64

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

dataset


<BatchDataset shapes: ((64, 100), (64, 100)), types: (tf.int64, tf.int64)>

### Build The Model

Use **tf.keras.Sequential** to define the model. 

For this simple example three layers are used to define our model:

    - tf.keras.layers.Embedding: The input layer.     
    A trainable lookup table that will map the numbers of each character to a vector with embedding_dim dimensions;
    
    - tf.keras.layers.GRU: A type of RNN with size units=rnn_units (You can also use a LSTM layer here.)

    - tf.keras.layers.Dense: The output layer, with vocab_size outputs.

In [14]:
# Length of the vocabulary in chars
vocab_size = len(vocab)

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

In [15]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
    model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim,
                              batch_input_shape=[batch_size, None]),
    tf.keras.layers.GRU(rnn_units,
                        return_sequences=True,
                        stateful=True,
                        recurrent_initializer='glorot_uniform'),
    tf.keras.layers.Dense(vocab_size)
  ])
    return model


In [16]:
model = build_model(
    vocab_size = len(vocab),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units,
    batch_size=BATCH_SIZE)


For each character the model looks up the embedding, runs the GRU one timestep with the embedding as input, and applies the dense layer to generate logits predicting the log-likelihood of the next character:
![sequential-model](https://www.tensorflow.org/tutorials/text/images/text_generation_training.png)

#### Please note that we choose to Keras sequential model here since all the layers in the model only have single input and produce single output. 
#### In case you want to retrieve and reuse the states from stateful RNN layer, you might want to build your model with Keras functional API or model subclassing. 
Please check Keras RNN guide for more details.


## Try the model

Now run the model to see that it behaves as expected.

First check the shape of the output:

In [17]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100, 65) # (batch_size, sequence_length, vocab_size)


In [18]:
#In the above example the sequence length of the input is 100 but the model can be run on inputs of any length:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (64, None, 256)           16640     
_________________________________________________________________
gru (GRU)                    (64, None, 1024)          3938304   
_________________________________________________________________
dense (Dense)                (64, None, 65)            66625     
Total params: 4,021,569
Trainable params: 4,021,569
Non-trainable params: 0
_________________________________________________________________


To get actual predictions from the model we need to sample from the output distribution, to get actual character indices. 

This distribution is defined by the logits over the character vocabulary.

**Note: It is important to sample from this distribution as taking the argmax of the distribution can easily get the model stuck in a loop.**

Try it for the first example in the batch:

In [19]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()

In [20]:
#This gives us, at each timestep, a prediction of the next character index:

sampled_indices

array([30, 64, 15, 14, 25, 12, 43, 46,  1, 23, 60, 35,  7, 29, 32,  5,  7,
       30, 40, 37, 41, 14, 13, 31, 60, 21, 56,  3, 28, 64, 16, 34,  6,  8,
       16,  0,  1, 45, 32, 52, 42, 15, 14, 33, 15, 25, 45,  8, 34, 38, 12,
        0, 48,  2, 47, 37, 40, 46,  4,  1, 27, 34, 58, 31, 33, 24, 11, 31,
       27,  3, 48, 64, 15, 54, 49, 44, 23, 37, 57,  2, 38,  6, 35, 37, 19,
       35, 63, 25, 22, 23, 11, 14, 34, 34, 20, 22, 19, 31, 44,  8])

In [21]:
#Decode these to see the text predicted by this untrained model:

print("Input: \n", repr("".join(idx2char[input_example_batch[0]])))
print()
print("Next Char Predictions: \n", repr("".join(idx2char[sampled_indices ])))

Input: 
 'UCESTER:\nHow fares our cousin, noble Lord of York?\n\nYORK:\nI thank you, gentle uncle. O, my lord,\nYou'

Next Char Predictions: 
 "RzCBM?eh KvW-QT'-RbYcBASvIr$PzDV,.D\n gTndCBUCMg.VZ?\nj!iYbh& OVtSUL;SO$jzCpkfKYs!Z,WYGWyMJK;BVVHJGSf."


### Train the model

At this point the problem can be treated as a standard classification problem. 

Given the previous RNN state, and the input this time step, predict the class of the next character.

Attach an optimizer, and a loss function

The standard **tf.keras.losses.sparse_categorical_crossentropy** loss function works in this case because it is applied across the last dimension of the predictions.

Because our model returns logits, we need to set the **from_logits** flag.

In [22]:
def loss(labels, logits):
    return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

example_batch_loss  = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("scalar_loss:      ", example_batch_loss.numpy().mean())


Prediction shape:  (64, 100, 65)  # (batch_size, sequence_length, vocab_size)
scalar_loss:       4.173931


Configure the training procedure using the **tf.keras.Model.compile** method. 

We'll use **tf.keras.optimizers.Adam** with default arguments and the loss function.

In [23]:
model.compile(optimizer='adam', loss=loss)

### Configure checkpoints

Use a **tf.keras.callbacks.ModelCheckpoint** to ensure that checkpoints are saved during training:

In [24]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'


# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)


### Execute the training

To keep training time reasonable, use 10 epochs to train the model. 

In Colab, set the runtime to GPU for faster training.

In [ ]:
EPOCHS=10

history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/10
172/172 [==============================] - 300s 2s/step - loss: 2.6724
Epoch 2/10
172/172 [==============================] - 347s 2s/step - loss: 1.9496
Epoch 3/10
172/172 [==============================] - 492s 3s/step - loss: 1.6832
Epoch 4/10
172/172 [==============================] - 494s 3s/step - loss: 1.5363
Epoch 5/10
172/172 [==============================] - 440s 3s/step - loss: 1.4497
Epoch 6/10
 94/172 [===============>..............] - ETA: 3:14 - loss: 1.3937

### Generate text

Restore the latest checkpoint

To keep this prediction step simple, use a batch size of 1.


Because of the way the RNN state is passed from timestep to timestep, the model only accepts a fixed batch size once built.


To run the model with a different batch_size, we need to rebuild the model and restore the weights from the checkpoint.

In [ ]:
tf.train.latest_checkpoint(checkpoint_dir)

In [ ]:
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)

model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))

model.build(tf.TensorShape([1, None]))

model.summary()

### The prediction loop

The following code block generates the text:

    - It Starts by choosing a start string, initializing the RNN state and setting the number of characters to generate.

    - Get the prediction distribution of the next character using the start string and the RNN state.

    - Then, use a categorical distribution to calculate the index of the predicted character. Use this predicted character as our next input to the model.

    - The RNN state returned by the model is fed back into the model so that it now has more context, instead than only one character. 
    After predicting the next character, the modified RNN states are again fed back into the model, which is how it learns as it gets more context from the previously predicted characters.


![model2](https://www.tensorflow.org/tutorials/text/images/text_generation_sampling.png)

Looking at the generated text, you'll see the model knows when to capitalize, make paragraphs and imitates a Shakespeare-like writing vocabulary.

With the small number of training epochs, it has not yet learned to form coherent sentences.

In [ ]:
def generate_text(model, start_string):
  # Evaluation step (generating text using the learned model)

  # Number of characters to generate
    num_generate = 1000

  # Converting our start string to numbers (vectorizing)
  input_eval = [char2idx[s] for s in start_string]
    input_eval = tf.expand_dims(input_eval, 0)

  # Empty string to store our results
  text_generated = []

  # Low temperatures results in more predictable text.
  # Higher temperatures results in more surprising text.
  # Experiment to find the best setting.
  temperature = 1.0

  # Here batch size == 1
  model.reset_states()
    for i in range(num_generate):
    predictions = model(input_eval)
    # remove the batch dimension
    predictions = tf.squeeze(predictions, 0)

    # using a categorical distribution to predict the character returned by the model
    predictions = predictions / temperature
    predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

    # We pass the predicted character as the next input to the model
    # along with the previous hidden state
    input_eval = tf.expand_dims([predicted_id], 0)

    text_generated.append(idx2char[predicted_id])

    return (start_string + ''.join(text_generated))


In [ ]:
print(generate_text(model, start_string=u"ROMEO: "))

### Print text for different characters:

In [ ]:
# star_words = [u"ROMEO: ", u"PUCK: ", u"MACBETH: ", u"HAMLET: ", u"IAGO: ", u"PROSPERO: ", u"LEAR: ", u"EDMUND: ", u"BENEDICK: "]
# for word in start_words:
#     filename = "outputs/{0}_out.txt".format(word[:-2])
#     print(word, filename)
#     out_text = open(filename, "w+")
#     out_text.write(generate_text(model, start_string = word))
#     out_text.close()


*The easiest thing you can do to improve the results it to train it for longer (try EPOCHS=30).*


You can also experiment with a different start string, or try adding another RNN layer to improve the model's accuracy, or adjusting the temperature parameter to generate more or less random predictions.
Advanced: Customized Training

The above training procedure is simple, but does not give you much control.


So now that you've seen how to run the model manually let's unpack the training loop, and implement it ourselves. This gives a starting point if, for example, to implement curriculum learning to help stabilize the model's open-loop output.


We will use **tf.GradientTape** to track the gradients. You can learn more about this approach by reading the eager execution guide.


The procedure works as follows:

    - First, reset the RNN state. We do this by calling the tf.keras.Model.reset_states method.

    - Next, iterate over the dataset (batch by batch) and calculate the predictions associated with each.

    - Open a tf.GradientTape, and calculate the predictions and loss in that context.

    - Calculate the gradients of the loss with respect to the model variables using the tf.GradientTape.grads method.

    - Finally, take a step downwards by using the optimizer's tf.train.Optimizer.apply_gradients method.


In [ ]:
model = build_model(
    vocab_size = len(vocab),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units,
    batch_size=BATCH_SIZE)

In [ ]:
optimizer = tf.keras.optimizers.Adam()

In [ ]:
@tf.function
def train_step(inp, target):
    with tf.GradientTape() as tape:
    predictions = model(inp)
    loss = tf.reduce_mean(
        tf.keras.losses.sparse_categorical_crossentropy(
            target, predictions, from_logits=True))
    grads = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(grads, model.trainable_variables))

    return loss

In [ ]:
# Training step
EPOCHS = 10

for epoch in range(EPOCHS):
    start = time.time()

  # resetting the hidden state at the start of every epoch
  model.reset_states()

    for (batch_n, (inp, target)) in enumerate(dataset):
    loss = train_step(inp, target)

    if batch_n % 100 == 0:
        template = 'Epoch {} Batch {} Loss {}'
        print(template.format(epoch+1, batch_n, loss))

  # saving (checkpoint) the model every 5 epochs
  if (epoch + 1) % 5 == 0:
    model.save_weights(checkpoint_prefix.format(epoch=epoch))

    print ('Epoch {} Loss {:.4f}'.format(epoch+1, loss))
    print ('Time taken for 1 epoch {} sec\n'.format(time.time() - start))

model.save_weights(checkpoint_prefix.format(epoch=epoch))
